#### Загрузим данные и посмотрим на них

In [1]:
!pip install matplotlib -q
!pip install pandas -q

!pip install "numpy<2"

import warnings
import matplotlib.pyplot as plt

from pandas import DataFrame

warnings.filterwarnings("ignore")

!pip install pandas -q

import pandas as pd
# Хак чтобы работало на Anaconda под MacOs 10.15.7 Catalina
pd.DataFrame.iteritems = pd.DataFrame.items

In [2]:
!pip install plotly -q

import plotly.graph_objects as go

def linear_plot(df, title):
    fig = go.Figure([go.Scatter(x=df['date'], y=df['close'], mode='lines')])
    fig.update_layout(plot_bgcolor='white',
                      xaxis_title='Date',
                      yaxis_title='Price',
                      title=title)
    fig.show()

def candlestick_plot(df, title):
    fig = go.Figure([go.Candlestick(x=df['date'],
                                open=df['open'],
                                high=df['high'],
                                low=df['low'],
                                close=df['close'])])
    fig.update_layout(xaxis_rangeslider_visible=False,
                      plot_bgcolor='white',
                      xaxis_title='Date',
                      yaxis_title='Price',
                      title=title)
    fig.update_yaxes(fixedrange=False)
    fig.show()

#### Настроим окружение

In [3]:
import os
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'

In [4]:
!pip install git+https://github.com/kernc/backtesting.py.git -q

from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


In [5]:
!pip install torch -q

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

#### Зафиксируем SEED

In [6]:
import random
import numpy as np

SEED = 777

def seed_everything(seed: int = 42) -> None:
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True)
    print(f"Using {seed} seed")

seed_everything(SEED)

Using 777 seed


In [7]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


### Загрузим данные

In [8]:
!pip install ccxt -q

import ccxt
import time

def get_data(symbol, timeframe, since):
    exchange = ccxt.binance()
    since = exchange.parse8601(since)
    all_ohlcvs = []

    while True:
        try:
            ohlcvs = exchange.fetch_ohlcv(symbol, timeframe, since)
            all_ohlcvs += ohlcvs
            if len(ohlcvs):
                print('Fetched', len(ohlcvs), symbol, timeframe, 'candles from', exchange.iso8601(ohlcvs[0][0]))
                since = ohlcvs[-1][0] + 1
                sleep_interval = exchange.rateLimit / 1000
                print('Sleep for', sleep_interval)
                time.sleep(sleep_interval)
            else:
                break
        except Exception as e:
            print(type(e).__name__, str(e))
    print('Fetched', len(all_ohlcvs), symbol, timeframe, 'candles in total')

    df = pd.DataFrame(all_ohlcvs)
    df.columns = ['date','open','high','low','close','volume']
    df = df.sort_values(by='date')
    df = df.drop_duplicates(subset='date').reset_index(drop=True)
    df['date'] = pd.to_datetime(df['date'], unit='ms')
    df.to_csv(symbol.replace('/', '_') + '_' + timeframe + '.csv', index=False)
    return df

In [9]:
symbol = 'ETH/USDT'
timeframe = '1d'
df = get_data(symbol, timeframe, '2020-01-01T00:00:00Z')
df

Fetched 500 ETH/USDT 1d candles from 2020-01-01T00:00:00.000Z
Sleep for 0.05
Fetched 500 ETH/USDT 1d candles from 2021-05-15T00:00:00.000Z
Sleep for 0.05
Fetched 500 ETH/USDT 1d candles from 2022-09-27T00:00:00.000Z
Sleep for 0.05
Fetched 337 ETH/USDT 1d candles from 2024-02-09T00:00:00.000Z
Sleep for 0.05
Fetched 1837 ETH/USDT 1d candles in total


,date,open,high,low,close,volume
0,2020-01-01,129.16,133.05,128.68,130.77,144770.52197
1,2020-01-02,130.72,130.78,126.38,127.19,213757.05806
2,2020-01-03,127.19,135.14,125.88,134.35,413055.18895
3,2020-01-04,134.37,135.85,132.50,134.20,184276.17102
4,2020-01-05,134.20,138.19,134.19,135.37,254120.45343
...,...,...,...,...,...,...
1832,2025-01-06,3636.00,3744.83,3610.63,3687.45,329642.36480
1833,2025-01-07,3687.44,3700.86,3356.31,3381.31,541543.28740
1834,2025-01-08,3381.31,3415.10,3208.20,3327.29,584749.96270
1835,2025-01-09,3327.29,3357.27,3158.00,3219.20,501818.42470


In [10]:
linear_plot(df, 'ETH/USDT')

In [11]:
df = pd.read_csv(symbol.replace('/', '_') + '_' + timeframe + '.csv')
df = df.sort_values(by='date')
df = df.drop_duplicates(subset='date').reset_index(drop=True)
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)
df

,open,high,low,close,volume
date,,,,,
2020-01-01,129.16,133.05,128.68,130.77,144770.52197
2020-01-02,130.72,130.78,126.38,127.19,213757.05806
2020-01-03,127.19,135.14,125.88,134.35,413055.18895
2020-01-04,134.37,135.85,132.50,134.20,184276.17102
2020-01-05,134.20,138.19,134.19,135.37,254120.45343
...,...,...,...,...,...
2025-01-06,3636.00,3744.83,3610.63,3687.45,329642.36480
2025-01-07,3687.44,3700.86,3356.31,3381.31,541543.28740
2025-01-08,3381.31,3415.10,3208.20,3327.29,584749.96270


In [12]:
df.shape

(1837, 5)

In [13]:
df.isnull().sum()

,0
open,0
high,0
low,0
close,0
volume,0


In [14]:
df.describe()

,open,high,low,close,volume
count,1837.000000,1837.000000,1837.000000,1837.000000,1.837000e+03
mean,1988.180996,2044.819864,1925.764692,1989.901747,6.440248e+05
std,1150.068238,1181.305270,1113.214327,1149.654111,4.901126e+05
min,107.670000,118.500000,86.000000,107.820000,5.851962e+04
25%,1253.220000,1291.000000,1211.670000,1254.250000,3.288816e+05
50%,1871.990000,1907.920000,1842.640000,1872.010000,5.081983e+05
75%,2920.990000,2984.520000,2817.000000,2921.730000,8.061430e+05
max,4807.980000,4868.000000,4713.890000,4807.980000,4.663240e+06


In [15]:
# Удалим значения где нет объемов
df = df.drop(df[df['volume']==0.0].index)

#### Преобразуем данные в формат, пригодный для бэктестинга

In [16]:
df = df.reset_index()
df.columns = df.columns.str.capitalize()
df.rename(columns={'Date': 'Datetime'}, inplace=True)
df["Datetime"] = pd.to_datetime(df["Datetime"])
df.set_index('Datetime', inplace=True)
df

,Open,High,Low,Close,Volume
Datetime,,,,,
2020-01-01,129.16,133.05,128.68,130.77,144770.52197
2020-01-02,130.72,130.78,126.38,127.19,213757.05806
2020-01-03,127.19,135.14,125.88,134.35,413055.18895
2020-01-04,134.37,135.85,132.50,134.20,184276.17102
2020-01-05,134.20,138.19,134.19,135.37,254120.45343
...,...,...,...,...,...
2025-01-06,3636.00,3744.83,3610.63,3687.45,329642.36480
2025-01-07,3687.44,3700.86,3356.31,3381.31,541543.28740
2025-01-08,3381.31,3415.10,3208.20,3327.29,584749.96270


#### Разобьем данные на обучающую и тестовую выборки

In [17]:
data = df.copy()
data

,Open,High,Low,Close,Volume
Datetime,,,,,
2020-01-01,129.16,133.05,128.68,130.77,144770.52197
2020-01-02,130.72,130.78,126.38,127.19,213757.05806
2020-01-03,127.19,135.14,125.88,134.35,413055.18895
2020-01-04,134.37,135.85,132.50,134.20,184276.17102
2020-01-05,134.20,138.19,134.19,135.37,254120.45343
...,...,...,...,...,...
2025-01-06,3636.00,3744.83,3610.63,3687.45,329642.36480
2025-01-07,3687.44,3700.86,3356.31,3381.31,541543.28740
2025-01-08,3381.31,3415.10,3208.20,3327.29,584749.96270


In [18]:
# Определяем дату начала тестовой выборки
test_start_date = data.index.max() - pd.DateOffset(months=6)

# Разделение данных на тренировочную и тестовую выборки по времени
train_data = data[data.index < test_start_date]
test_data = data[data.index >= test_start_date]

print(f"Train size: {len(train_data)}, Test size: {len(test_data)}")

Train size: 1652, Test size: 185


#### Рассмотрим baseline стратегию

In [19]:
class Baseline(Strategy):
    n1 = 10
    n2 = 20

    def init(self):
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)

    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()

In [20]:
def run_backtest(test_data, strategy):
  bt = Backtest(test_data, Baseline, cash=1_000_000, commission=.002)
  stats = bt.run()
  bt.plot()
  print(stats)


In [21]:
run_backtest(test_data, Baseline)

Start                     2024-07-10 00:00:00
End                       2025-01-10 00:00:00
Duration                    184 days 00:00:00
Exposure Time [%]                   88.648649
Equity Final [$]                 819125.19684
Equity Peak [$]                   1283601.688
Return [%]                          -18.08748
Buy & Hold Return [%]                5.770949
Return (Ann.) [%]                  -32.540603
Volatility (Ann.) [%]               43.362955
Sharpe Ratio                        -0.750424
Sortino Ratio                       -0.769262
Calmar Ratio                        -0.899274
Max. Drawdown [%]                  -36.185407
Avg. Drawdown [%]                  -18.597817
Max. Drawdown Duration      156 days 00:00:00
Avg. Drawdown Duration       79 days 00:00:00
# Trades                                   10
Win Rate [%]                             30.0
Best Trade [%]                       19.61509
Worst Trade [%]                    -12.872496
Avg. Trade [%]                    

На простой стратегии скользящих средних получили убыток

In [22]:
class CFG:
    INPUT_SIZE=1
    HIDDEN_LAYER_SIZE=50
    OUTPUT_SIZE=1
    LEARNING_RATE=0.01
    BATCH_SIZE=128
    EPOCHS=40
    WINDOW=30

In [23]:
!pip install scikit-learn -q

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled_train = scaler.fit_transform(train_data[["Close"]])
scaled_test = scaler.transform(test_data[["Close"]])

In [24]:
scaled_train

array([[0.00488281],
       [0.00412114],
       [0.00564449],
       ...,
       [0.60065615],
       [0.61938104],
       [0.62951687]])

In [25]:
scaled_test

array([[0.63683577],
       [0.63652088],
       [0.64382064],
       [0.65276714],
       [0.6674794 ],
       [0.71818193],
       [0.70982903],
       [0.69768476],
       [0.70607809],
       [0.72246689],
       [0.72543913],
       [0.72935815],
       [0.70886523],
       [0.71799471],
       [0.686783  ],
       [0.6526714 ],
       [0.67376217],
       [0.66831555],
       [0.67281539],
       [0.68292143],
       [0.67474086],
       [0.66485396],
       [0.65861162],
       [0.61312594],
       [0.59483507],
       [0.54915152],
       [0.49184921],
       [0.50072976],
       [0.47551147],
       [0.54778561],
       [0.52997345],
       [0.53234358],
       [0.52073972],
       [0.5562534 ],
       [0.55202802],
       [0.54330704],
       [0.52382685],
       [0.52868626],
       [0.53332014],
       [0.53281803],
       [0.53796892],
       [0.52445023],
       [0.53676683],
       [0.53510093],
       [0.56480205],
       [0.56597648],
       [0.56132344],
       [0.547

In [26]:
def create_sequences(data: np.ndarray, seq_length: int=50) -> tuple[np.ndarray]:
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length, 0])
    return np.array(X), np.array(y)

In [27]:
X_train, y_train = create_sequences(scaled_train, CFG.WINDOW)
X_test, y_test = create_sequences(scaled_test, CFG.WINDOW)

In [28]:
X_train

array([[[0.00488281],
        [0.00412114],
        [0.00564449],
        ...,
        [0.0144293 ],
        [0.0140208 ],
        [0.01635476]],

       [[0.00412114],
        [0.00564449],
        [0.00561257],
        ...,
        [0.0140208 ],
        [0.01635476],
        [0.0153548 ]],

       [[0.00564449],
        [0.00561257],
        [0.0058615 ],
        ...,
        [0.01635476],
        [0.0153548 ],
        [0.01612286]],

       ...,

       [[0.75965499],
        [0.76034646],
        [0.76562926],
        ...,
        [0.62803819],
        [0.61146004],
        [0.62955516]],

       [[0.76034646],
        [0.76562926],
        [0.75742741],
        ...,
        [0.61146004],
        [0.62955516],
        [0.60065615]],

       [[0.76562926],
        [0.75742741],
        [0.72114779],
        ...,
        [0.62955516],
        [0.60065615],
        [0.61938104]]])

In [29]:
class LSTM(nn.Module):
    def __init__(self, input_dim: int, hidden_dim: int, output_dim: int, layer_num: int=3) -> None:
        super().__init__()
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.lstm = torch.nn.LSTM(
            input_dim,
            hidden_dim,
            layer_num,
            batch_first=True
        )
        self.dr = torch.nn.Dropout2d(0.1)
        self.fc = torch.nn.Linear(
            hidden_dim,
            output_dim
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        lstm_out, (hn, cn) = self.lstm(x)
        out = self.fc(lstm_out[:, -1])
        return out

In [30]:
lstm_model = LSTM(CFG.INPUT_SIZE, CFG.HIDDEN_LAYER_SIZE, CFG.OUTPUT_SIZE).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(lstm_model.parameters(), lr=CFG.LEARNING_RATE)

In [31]:
def worker_init_fn(worker_id: int) -> None:
    np.random.seed(CFG.SEED + worker_id)

train_data_pt = TensorDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train).float())
full_train_loader = DataLoader(
    train_data_pt,
    batch_size=CFG.BATCH_SIZE,
    shuffle=True,
    num_workers=0,
    worker_init_fn=worker_init_fn
    )

In [32]:
lstm_model.train()

for epoch in range(CFG.EPOCHS):
    for X_batch, y_batch in full_train_loader:
        optimizer.zero_grad()
        y_pred = lstm_model(X_batch.to(device))
        loss = criterion(y_pred, y_batch.to(device).view(-1, 1))
        loss.backward()
        optimizer.step()
    if (epoch + 1) % 2 == 0:
        print(f"Epoch {epoch + 1}/{CFG.EPOCHS}, Loss: {loss.item()}")

Epoch 2/40, Loss: 0.010650087147951126
Epoch 4/40, Loss: 0.0014610744547098875
Epoch 6/40, Loss: 0.0008614597609266639
Epoch 8/40, Loss: 0.0012823233846575022
Epoch 10/40, Loss: 0.0006744057754985988
Epoch 12/40, Loss: 0.000524599920026958
Epoch 14/40, Loss: 0.0011336266761645675
Epoch 16/40, Loss: 0.0007500085630454123
Epoch 18/40, Loss: 0.000405962549848482
Epoch 20/40, Loss: 0.0005059555405750871
Epoch 22/40, Loss: 0.0006794806686230004
Epoch 24/40, Loss: 0.00030215721926651895
Epoch 26/40, Loss: 0.00026959998649545014
Epoch 28/40, Loss: 0.0002538647677283734
Epoch 30/40, Loss: 0.0005795691395178437
Epoch 32/40, Loss: 0.0005614387919194996
Epoch 34/40, Loss: 0.0005098194815218449
Epoch 36/40, Loss: 0.0003549249086063355
Epoch 38/40, Loss: 0.0010909445118159056
Epoch 40/40, Loss: 0.0003575959417503327


In [33]:
lstm_model.eval()

with torch.inference_mode():
    X_test_tensor = torch.from_numpy(X_test).to(device).float()
    predicted_close_prices = lstm_model(X_test_tensor).cpu().numpy()
predicted_close_prices = scaler.inverse_transform(predicted_close_prices)

predicted_close_prices[:5]

array([[2614.117 ],
       [2627.193 ],
       [2626.7502],
       [2580.489 ],
       [2706.7961]], dtype=float32)

In [34]:
y_test

array([0.52997345, 0.53234358, 0.52073972, 0.5562534 , 0.55202802,
       0.54330704, 0.52382685, 0.52868626, 0.53332014, 0.53281803,
       0.53796892, 0.52445023, 0.53676683, 0.53510093, 0.56480205,
       0.56597648, 0.56132344, 0.54735796, 0.49987873, 0.5149846 ,
       0.51483141, 0.51448887, 0.51172513, 0.49315342, 0.5170441 ,
       0.49306194, 0.49847026, 0.48104533, 0.45049743, 0.46078431,
       0.46583095, 0.47906454, 0.4852388 , 0.47503276, 0.47954538,
       0.49601928, 0.49146625, 0.46983081, 0.46548628, 0.47529871,
       0.48230911, 0.50155527, 0.52202053, 0.53287122, 0.5261906 ,
       0.54022629, 0.54155178, 0.5259672 , 0.53709661, 0.55032382,
       0.5462346 , 0.54249217, 0.53070747, 0.49784901, 0.48004323,
       0.47700078, 0.49074712, 0.49080031, 0.496198  , 0.49251302,
       0.49638097, 0.48139851, 0.4848069 , 0.49608524, 0.50393604,
       0.50234247, 0.5365711 , 0.53180956, 0.53259464, 0.53146701,
       0.53920505, 0.54048798, 0.5614894 , 0.54442402, 0.53508

In [35]:
scaler.inverse_transform(y_test.reshape(-1, 1))[:5]

array([[2598.78],
       [2609.92],
       [2555.38],
       [2722.3 ],
       [2702.44]])

In [36]:
def create_sequences_for_strategy(data: np.ndarray, seq_length: int=50) -> tuple[np.ndarray]:
    X = []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
    return np.array(X)

In [37]:
class LstmStrategy(Strategy):
    def init(self) -> None:
        self.scaler = scaler
        self.model = lstm_model
        self.model.eval()

        self.forecasts = self.I(lambda: np.repeat(np.nan, len(self.data)), name='forecast')

    def next(self) -> None:
        if len(self.data) <= CFG.WINDOW:
            return

        scaled_prices = self.scaler.transform(self.data.df[["Close"]])[-CFG.WINDOW:].reshape(-1, 1)
        with torch.inference_mode():
            tensor = torch.from_numpy(scaled_prices).to(device).float().view(1, CFG.WINDOW, 1)
            predicted_close_price_scaled = self.model(tensor).cpu().numpy()
        predicted_close_price = self.scaler.inverse_transform(predicted_close_price_scaled)[0][0]

        self.forecasts[-1] = predicted_close_price

        if self.data.Close / predicted_close_price < 0.99:
            self.position.close()
            self.buy()

        elif self.data.Close / predicted_close_price > 1.01:
            self.position.close()
            self.sell()

In [38]:
run_backtest(test_data, LstmStrategy)

Start                     2024-07-10 00:00:00
End                       2025-01-10 00:00:00
Duration                    184 days 00:00:00
Exposure Time [%]                   88.648649
Equity Final [$]                 819125.19684
Equity Peak [$]                   1283601.688
Return [%]                          -18.08748
Buy & Hold Return [%]                5.770949
Return (Ann.) [%]                  -32.540603
Volatility (Ann.) [%]               43.362955
Sharpe Ratio                        -0.750424
Sortino Ratio                       -0.769262
Calmar Ratio                        -0.899274
Max. Drawdown [%]                  -36.185407
Avg. Drawdown [%]                  -18.597817
Max. Drawdown Duration      156 days 00:00:00
Avg. Drawdown Duration       79 days 00:00:00
# Trades                                   10
Win Rate [%]                             30.0
Best Trade [%]                       19.61509
Worst Trade [%]                    -12.872496
Avg. Trade [%]                    

Получили также убыток

### Попробуем добавить Bagging

In [39]:
def create_train_loader(data: pd.DataFrame, start: str, end: str, scaler: MinMaxScaler, batch_size: int) -> DataLoader:
    train_data = data[(data.index >= start) & (data.index < end)]
    scaled_train_data = scaler.transform(train_data[["Close"]])
    X_train, y_train = create_sequences(scaled_train_data, CFG.WINDOW)
    train_data_pt = TensorDataset(
        torch.from_numpy(X_train).to(device).float(),
        torch.from_numpy(y_train).to(device).float()
        )
    train_loader = DataLoader(
        train_data_pt,
        batch_size=batch_size,
        shuffle=True,
        num_workers=0,
        worker_init_fn=worker_init_fn
        )
    return train_loader

In [40]:
def train_model(
        model: nn.Module,
        criterion: nn.MSELoss,
        optimizer: torch.optim.Adam,
        train_loader: DataLoader,
        epochs: int
        ) -> None:
    model.train()
    for epoch in range(epochs):
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch.view(-1, 1))
            loss.backward()
            optimizer.step()
    model.eval()

In [41]:
scaler = MinMaxScaler()
_ = scaler.fit(data[data.index < test_start_date][["Close"]])

In [42]:
bagging_models = [LSTM(CFG.INPUT_SIZE, CFG.HIDDEN_LAYER_SIZE, CFG.OUTPUT_SIZE).to(device) for _ in range(4)]
time_range = [
    ("2020-01-01", "2021-01-01"),
    ("2021-01-01", "2022-01-01"),
    ("2022-01-01", "2023-01-01"),
    ("2023-01-01", str(test_start_date)),
    ]
assert len(bagging_models) == len(time_range)

In [43]:
!pip install tqdm -q

from tqdm.auto import tqdm

for idx, one_model in tqdm(enumerate(bagging_models), total=len(bagging_models)):
    train_loader = create_train_loader(data, time_range[idx][0], time_range[idx][1], scaler, CFG.BATCH_SIZE)
    train_model(
        one_model,
        nn.MSELoss(),
        torch.optim.Adam(one_model.parameters(), lr=CFG.LEARNING_RATE),
        train_loader,
        CFG.EPOCHS
        )

  0%|          | 0/4 [00:00<?, ?it/s]

In [44]:
class Bagging_LstmStrategy(Strategy):
    def init(self) -> None:
        self.scaler = scaler
        self.models = bagging_models
        self.models = [model.eval() for model in self.models]

        # Индикатор для отображения предсказаний модели на графике
        self.forecasts = self.I(lambda: np.repeat(np.nan, len(self.data)), name='forecast')

    def next(self) -> None:
        if len(self.data) <= CFG.WINDOW:
            return

        scaled_prices = self.scaler.transform(self.data.df[["Close"]])[-CFG.WINDOW:].reshape(-1, 1)

        predicted_close_prices = []

        for one_model in self.models:
            with torch.inference_mode():
                tensor = torch.from_numpy(scaled_prices).to(device).float().view(1, CFG.WINDOW, 1)
                predicted_close_price_scaled = one_model(tensor).cpu().numpy()
            predicted_close_price = self.scaler.inverse_transform(predicted_close_price_scaled)[0][0]
            predicted_close_prices.append(predicted_close_price)

        final_predicted_close_price = np.mean(predicted_close_prices)

        self.forecasts[-1] = final_predicted_close_price

        if self.data.Close / predicted_close_price < 0.99:
            self.position.close()
            self.buy()

        elif self.data.Close / predicted_close_price > 1.01:
            self.position.close()
            self.sell()

In [45]:
run_backtest(test_data, Bagging_LstmStrategy)

Start                     2024-07-10 00:00:00
End                       2025-01-10 00:00:00
Duration                    184 days 00:00:00
Exposure Time [%]                   88.648649
Equity Final [$]                 819125.19684
Equity Peak [$]                   1283601.688
Return [%]                          -18.08748
Buy & Hold Return [%]                5.770949
Return (Ann.) [%]                  -32.540603
Volatility (Ann.) [%]               43.362955
Sharpe Ratio                        -0.750424
Sortino Ratio                       -0.769262
Calmar Ratio                        -0.899274
Max. Drawdown [%]                  -36.185407
Avg. Drawdown [%]                  -18.597817
Max. Drawdown Duration      156 days 00:00:00
Avg. Drawdown Duration       79 days 00:00:00
# Trades                                   10
Win Rate [%]                             30.0
Best Trade [%]                       19.61509
Worst Trade [%]                    -12.872496
Avg. Trade [%]                    

Наконец-то получили прибыль

### TSMixer

In [46]:
class TSMIXER_CFG:
    WINDOW = 80
    BATCH_SIZE = 128
    LR = 1e-5
    SEED = 777

In [47]:
from sklearn.preprocessing import StandardScaler

!pip install tensorflow -q

import tensorflow as tf
from tensorflow.keras import layers

class DataLoader:
    def __init__(self, batch_size, seq_len, pred_len, data):
        self.batch_size = batch_size
        self.seq_len = seq_len
        self.pred_len = pred_len
        self.target_slice = slice(0, None)

        self._read_data(data)

    def _read_data(self, df):
        train_df = df[df.index < "2024-01-01"]
        val_df = df[(df.index >= "2024-01-01") & (df.index < test_start_date)]
        test_df = df[df.index >= test_start_date]

        # standardize by training set
        self.scaler = StandardScaler()
        self.scaler.fit(train_df.values)

        def scale_df(df, scaler):
            data = scaler.transform(df.values)
            return pd.DataFrame(data, index=df.index, columns=df.columns)

        self.train_df = scale_df(train_df, self.scaler)
        self.val_df = scale_df(val_df, self.scaler)
        self.test_df = scale_df(test_df, self.scaler)
        self.n_feature = self.train_df.shape[-1]

    def _split_window(self, data):
        inputs = data[:, : self.seq_len, :]
        labels = data[:, self.seq_len :, self.target_slice]

        inputs.set_shape([None, self.seq_len, None])
        labels.set_shape([None, self.pred_len, None])
        return inputs, labels

    def _make_dataset(self, data, shuffle=True):
        data = np.array(data, dtype=np.float32)
        ds = tf.keras.utils.timeseries_dataset_from_array(
            data=data,
            targets=None,
            sequence_length=(self.seq_len + self.pred_len),
            sequence_stride=1,
            shuffle=shuffle,
            batch_size=self.batch_size,
        )
        ds = ds.map(self._split_window)
        return ds

    def inverse_transform(self, data):
        return self.scaler.inverse_transform(data)

    def get_train(self, shuffle: bool=True):
        return self._make_dataset(self.train_df, shuffle=shuffle)

    def get_val(self):
        return self._make_dataset(self.val_df, shuffle=False)

    def get_test(self):
        return self._make_dataset(self.test_df, shuffle=False)

In [48]:
data_loader = DataLoader(
    batch_size=TSMIXER_CFG.BATCH_SIZE,
    seq_len=TSMIXER_CFG.WINDOW,
    pred_len=1,
    data=data[["Close"]]
    )

tf_train_data = data_loader.get_train()
tf_val_data = data_loader.get_val()
tf_test_data = data_loader.get_test()

In [49]:
class TransposeLayer(layers.Layer):
    def __init__(self, perm: list[int], **kwargs) -> None:
        super().__init__(**kwargs)
        self.perm = perm

    def call(self, inputs):
        return tf.transpose(inputs, perm=self.perm)

In [50]:
def res_block(inputs, ff_dim: int):
    norm = layers.LayerNormalization

    # Time mixing
    x = norm(axis=[-2, -1])(inputs)
    x = TransposeLayer(perm=[0, 2, 1])(x)  # [Batch, Channel, Input Length]
    x = layers.Dense(x.shape[-1], activation='relu')(x)
    x = TransposeLayer(perm=[0, 2, 1])(x)  # [Batch, Input Length, Channel]
    x = layers.Dropout(0.7)(x)
    res = x + inputs

    # Feature mixing
    x = norm(axis=[-2, -1])(res)
    x = layers.Dense(ff_dim, activation='relu')(x)  # [Batch, Input Length, FF_Dim]
    x = layers.Dropout(0.7)(x)
    x = layers.Dense(inputs.shape[-1])(x)  # [Batch, Input Length, Channel]
    x = layers.Dropout(0.7)(x)
    return x + res

In [51]:
def build_model(
    input_shape: tuple[int],
    pred_len: int,
    n_block: int,
    ff_dim: int,
    target_slice,
):
    inputs = tf.keras.Input(shape=input_shape)
    x = inputs  # [Batch, Input Length, Channel]
    for _ in range(n_block):
        x = res_block(x, ff_dim)

    if target_slice:
        x = x[:, :, target_slice]

    # Temporal projection
    x = TransposeLayer(perm=[0, 2, 1])(x)  # [Batch, Channel, Input Length]
    x = layers.Dense(pred_len)(x)  # [Batch, Channel, Output Length]
    outputs = TransposeLayer(perm=[0, 2, 1])(x)  # [Batch, Output Length, Channel])
    return tf.keras.Model(inputs, outputs)

In [52]:
model = build_model(
    input_shape=(TSMIXER_CFG.WINDOW, data_loader.n_feature),
    pred_len=1,
    n_block=8,
    ff_dim=TSMIXER_CFG.WINDOW,
    target_slice=data_loader.target_slice
)

In [53]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 80, 1)          │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization       │ (None, 80, 1)          │            160 │ input_layer[0][0]      │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ transpose_layer           │ (None, 1, 80)          │              0 │ layer_normalization[0… │
│ (TransposeLayer)          │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 1, 80)          │          6,480 │ transpose_layer[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ transpose_layer_1         │ (None, 80, 1)          │              0 │ dense[0][0]            │
│ (TransposeLayer)          │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 80, 1)          │              0 │ transpose_layer_1[0][… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add (Add)                 │ (None, 80, 1)          │              0 │ dropout[0][0],         │
│                           │                        │                │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_1     │ (None, 80, 1)          │            160 │ add[0][0]              │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 80, 80)         │            160 │ layer_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 80, 80)         │              0 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 80, 1)          │             81 │ dropout_1[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_2 (Dropout)       │ (None, 80, 1)          │              0 │ dense_2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_1 (Add)               │ (None, 80, 1)          │              0 │ dropout_2[0][0],       │
│                           │                        │                │ add[0][0]              │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_2     │ (None, 80, 1)          │            160 │ add_1[0][0]            │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ transpose_layer_2         │ (None, 1, 80)          │              0 │ layer_normalization_2… │
│ (TransposeLayer)          │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)      

 Total params: 56,409 (220.35 KB)

 Trainable params: 56,409 (220.35 KB)

 Non-trainable params: 0 (0.00 B)

In [54]:
tf.keras.utils.set_random_seed(TSMIXER_CFG.SEED)

optimizer = tf.keras.optimizers.Adam(TSMIXER_CFG.LR)

model.compile(optimizer, loss='mse', metrics=['mae', 'mse'])

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='tsmixer_checkpoints/model.weights.h5',
    verbose=1,
    save_best_only=True,
    save_weights_only=True
)

early_stop_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5
)

In [ ]:
history = model.fit(
    tf_train_data,
    epochs=500,
    validation_data=tf_val_data,
    callbacks=[checkpoint_callback, early_stop_callback]
)

Epoch 1/500
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - loss: 40.6595 - mae: 5.1480 - mse: 40.6595 
Epoch 1: val_loss improved from inf to 28.06126, saving model to tsmixer_checkpoints/model.weights.h5
11/11 ━━━━━━━━━━━━━━━━━━━━ 106s 4s/step - loss: 40.6678 - mae: 5.1515 - mse: 40.6678 - val_loss: 28.0613 - val_mae: 4.9964 - val_mse: 28.0613
Epoch 2/500
 7/11 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 39.2757 - mae: 5.0121 - mse: 39.2757  
Epoch 2: val_loss improved from 28.06126 to 27.05908, saving model to tsmixer_checkpoints/model.weights.h5
11/11 ━━━━━━━━━━━━━━━━━━━━ 14s 35ms/step - loss: 39.5754 - mae: 5.0395 - mse: 39.5754 - val_loss: 27.0591 - val_mae: 4.8982 - val_mse: 27.0591
Epoch 3/500
 6/11 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 42.2149 - mae: 5.1132 - mse: 42.2149 
Epoch 3: val_loss improved from 27.05908 to 26.07945, saving model to tsmixer_checkpoints/model.weights.h5
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 41.1258 - mae: 5.0865 - mse: 41.1258 - val_loss: 26.0795 - 

In [ ]:
print(f"Лучшая эпоха: {np.argmin(history.history['val_loss'])}")

In [ ]:
model.load_weights("tsmixer_checkpoints/model.weights.h5")

In [ ]:
predictions = model.predict(tf_test_data)

scaled_preds = predictions[-1,:,:]

preds = data_loader.inverse_transform(scaled_preds)
preds_df = pd.DataFrame(preds)
preds_df

In [ ]:
class TSMixer(Strategy):
    def init(self) -> None:
        self.data_loader = data_loader
        self.model = model

        self.forecasts = self.I(lambda: np.repeat(np.nan, len(self.data)), name='forecast')

    def next(self) -> None:
        if len(self.data) <= TSMIXER_CFG.WINDOW+1:
            return

        scaled_prices = self.data_loader._make_dataset(
            self.data_loader.scaler.transform(self.data.df[["Close"]][-(TSMIXER_CFG.WINDOW+1):]),
            shuffle=False
            )

        predicted_close_price = self.data_loader.inverse_transform(
            self.model.predict(scaled_prices, verbose=0)[-1,:,:]
            )[0][0]

        self.forecasts[-1] = predicted_close_price

        if self.data.Close / predicted_close_price < 0.99:
            self.position.close()
            self.buy()

        elif self.data.Close / predicted_close_price > 1.01:
            self.position.close()
            self.sell()

In [ ]:
run_backtest(tf_test_data, TSMixer)

In [ ]:
class TSMixerEnsemble(Strategy):
    n1 = 5
    n2 = 10

    def init(self) -> None:
        self.data_loader = data_loader
        self.model = model
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)

        self.forecasts = self.I(lambda: np.repeat(np.nan, len(self.data)), name='forecast')

    def next(self) -> None:
        if len(self.data) <= TSMIXER_CFG.WINDOW+1:
            return

        scaled_prices = self.data_loader._make_dataset(
            self.data_loader.scaler.transform(self.data.df[["Close"]][-(TSMIXER_CFG.WINDOW+1):]),
            shuffle=False
            )

        predicted_close_price = self.data_loader.inverse_transform(
            self.model.predict(scaled_prices, verbose=0)[-1,:,:]
            )[0][0]

        self.forecasts[-1] = predicted_close_price

        if crossover(self.sma1, self.sma2) and (self.data.Close / predicted_close_price < 0.99):
            self.position.close()
            self.buy()

        elif crossover(self.sma2, self.sma1) and (self.data.Close / predicted_close_price > 1.01):
            self.position.close()
            self.sell()

In [ ]:
run_backtest(tf_test_data, TSMixerEnsemble)